# Challenge

Another approach to identifying fraudulent transactions is to look for outliers in the data. Standard deviation or quartiles are often used to detect outliers. Using this starter notebook, code two Python functions:

* One that uses standard deviation to identify anomalies for any cardholder.

* Another that uses interquartile range to identify anomalies for any cardholder.

## Identifying Outliers using Standard Deviation

In [1]:
# Initial imports
import pandas as pd
import numpy as np
import random
import psycopg2
from dotenv import load_dotenv
import os


In [2]:
# Create a connection to the database
load_dotenv()
pgpass = os.getenv('pgpass')
conn = psycopg2.connect(database = 'suspicious_transactions', user = 'postgres', password = pgpass, port = 5432)



In [10]:
# Write function that locates outliers using standard deviation
def std_outlier(id):
    query = f'''
    select date, amount
    from full_data
    where cardholder_id = {id}
    order by date;
    '''
    df = pd.read_sql(query, conn).set_index('date')
    upper_range = (df['amount'].std() * 2) + df['amount'].median()
    outlier_df = df[df['amount'] > upper_range]
    
    print(f"The median charge is ${round(df['amount'].median(), 2)}")
    print(f"The upper range of charges starts at ${round(upper_range, 2)}")
    print(f"There are {outlier_df.count()[0]} outliers for cardholder id #{id}")
    return outlier_df

In [11]:
# Find anomalous transactions for 3 random card holders
std_outlier(2)

The median charge is $10.26
The upper range of charges starts at $21.34
There are 0 outliers for cardholder id #2


,amount
date,


In [12]:
std_outlier(18)

The median charge is $10.37
The upper range of charges starts at $659.8
There are 7 outliers for cardholder id #18


,amount
date,
2018-02-19 22:48:25,1839.0
2018-04-03 03:23:37,1077.0
2018-06-03 20:02:28,1814.0
2018-07-18 09:19:08,974.0
2018-09-10 22:49:41,1176.0
2018-11-17 05:30:43,1769.0
2018-12-13 12:09:58,1154.0


In [13]:
std_outlier(25)

The median charge is $10.14
The upper range of charges starts at $627.58
There are 9 outliers for cardholder id #25


,amount
date,
2018-01-30 18:31:00,1177.0
2018-03-06 07:18:09,1334.0
2018-04-08 06:03:50,1063.0
2018-05-13 06:31:20,1046.0
2018-06-04 03:46:15,1162.0
2018-06-06 21:50:17,749.0
2018-06-22 06:16:50,1813.0
2018-08-16 10:01:00,1001.0
2018-12-18 13:33:37,1074.0


## Identifying Outliers Using Interquartile Range

In [5]:
# Write a function that locates outliers using interquartile range
def iqr_outlier(id):
    query = f'''
    select date, amount
    from full_data
    where cardholder_id = {id}
    order by date;
    '''
    df = pd.read_sql(query, conn).set_index('date')
    q3 = np.median(df['amount'][5:])
    q1, q3 = np.percentile(df['amount'], [25, 75])
    iqr = q3 - q1
    upper_range = q3 + (1.5 * iqr)
    outlier_df = df[df['amount'] > upper_range]
    
    print(f"the median charge is ${round(df['amount'].median(), 2)}")
    print(f"the upper range of charges starts at ${round(upper_range, 2)}")
    print(f"there are {outlier_df.count()[0]} outliers for cardholder id #{id}")
    outlier_df = df[df['amount'] > upper_range]
    return outlier_df

In [7]:
# Find anomalous transactions for 3 random card holders
iqr_outlier(2)

the median charge is $10.26
the upper range of charges starts at $27.03
there are 0 outliers for cardholder id #2


,amount
date,


In [14]:
iqr_outlier(18)

the median charge is $10.37
the upper range of charges starts at $35.21
there are 12 outliers for cardholder id #18


,amount
date,
2018-01-07 01:10:54,175.0
2018-01-08 11:15:36,333.0
2018-02-19 22:48:25,1839.0
2018-04-03 03:23:37,1077.0
2018-06-03 20:02:28,1814.0
2018-06-30 01:56:19,121.0
2018-07-06 16:12:08,117.0
2018-07-18 09:19:08,974.0
2018-09-02 11:20:42,458.0


In [15]:
iqr_outlier(25)

the median charge is $10.14
the upper range of charges starts at $31.58
there are 12 outliers for cardholder id #25


,amount
date,
2018-01-30 18:31:00,1177.0
2018-03-06 07:18:09,1334.0
2018-04-01 07:17:21,100.0
2018-04-08 06:03:50,1063.0
2018-04-09 18:28:25,269.0
2018-05-13 06:31:20,1046.0
2018-06-04 03:46:15,1162.0
2018-06-06 21:50:17,749.0
2018-06-22 06:16:50,1813.0
